# ROS Basics in 5 Days

## Course Project

<img src="ROSIn5Days.png" width="200"/>

### In this rosject, you will have to apply all that you have learned throughout the ROS Basics course to a real robot.

You will practice with a simulation and a real robot.

The real robot is a Turtlebot3, which is running in Barcelona, Spain. You will connect remotely to it and practice on it.

# How to proceed

This rosject is composed of three parts. You should do each part when the **ROS Basics in 5 Days** course indicates. 

* **PART I**: practice about topic publishers and subscribers
* **PART II**: practice about services
* **PART III**: practice about actions

Go to the part that the course indicates to you and try to finish it. But before doing that, **launch the simulation** of the project. You will use the simulation to practice with the simulated environment.

## Try in simulation first

You should use the simulation to test your code while you are trying to get the exercises of this project done. 

<img src="new_sim.png"/>

To launch the simulation, do the following:

1. Open a terminal by pressing on the *terminal icon* at the bottom left.

<img src="term.png"/>

2. On the opened terminal, type the following ROS command that launches the simulation

To see the simulation, press the Gazebo button in the bottom left side of your screen, and select *Open Gazebo*

<img src="gazebo.png"/>

**Wait around 30 seconds at the most** for the simulation to start and the simulation should appear in the Gazebo window.

# PART I: Topics

In this part, you are going to practice how to use topics to control a robot. Your goal is to create a ROS program that makes the robot have a wall following behavior.

### Wall following behavior

The wall following behavior is a behavior that makes the robot follow along the wall on its right hand side. This means that the robot must be moving forward at a 30cm (1 foot) distance from the wall, having the wall on its right hand side, the entire time.

To achieve this behavior in the robot, you need to do two things:

#### Subscribe to the laser topic of the robot

You need to subscribe to the laser topic and capture the rays. Select the ray on the right (the one that makes a 90º angle to the right with the front of the robot) and use it to measure the robot's distance to the wall.

* If the ray distance is **bigger than 0.3m**, you need to make the robot approach the wall a little, by adding some rotational speed to the robot
* If the ray distance is **lower than 0.2m**, you need to move the robot away from the wall, by adding rotational speed in the opposite direction
* If the ray distance is **between 0.2m and 0.3m**, just keep the robot moving forward

**IMPORTANT**

When the robot is moving along a wall, it can reach the next wall crossing its way. At that point in time, you should include a behavior that progressively transitions the robot from following the current wall to the next one.

For that, we recommend that you use the front laser ray. If the distance measured by that ray is shorter than 0.5m, then make the robot turn fast to the left (moving forward at the same time).

The result of this behavior must be that the robot moves along the whole environment (see video below).

### Test your wall following program on the simulation

Create the wall following program and test it on the simulation. If it doesn't work on the simulation, you cannot try it on the real robot. 

**If the program doesn't work on the simulation, it is 100% not going to work on the real robot.**

Real life robot development works this way: first try it on a simulation. When it works there, then you try it on the real one. Never the other way around.

<img src="wall_follow_long.gif"/>

To test on the simulation:

1. Make sure the simulation is launched as explained above
2. On another terminal, launch the keyboard teleop with the following command:

3. Now use the keys to move the robot to a convenient position to test your ROS program. This means, close to a wall looking along it. Remember that to be able to move the robot with the keys, **the terminal where you launched the teleop has to have the focus**.

**IMPORTANT**: Close this *teleop* program once you have the proper position. Otherwise, it will interfere with your own program when you launch it.

4. Now launch your ROS program and see the results. If the robot doesn't behave correctly, debug your program and try to figure out the reason.

### If working in simulation, then go to the real robot

Once the robot is moving along the walls of the simulated environment, it is time to test it on the real robot.

The steps are:
1. Book a date and time in the *Real Robot Lab* to test your program (see Appendix below that teaches how to do that).
2. On the day and time selected, open this rosject
3. Connect to the real robot lab from within this rosject (see Appendix below)
4. Launch the keyboard program and move the real robot to the proper position to start when you launch your program.
5. Launch your program and see if it works properly. Chances are it will not because a simulation is not an exact copy of the real environment. Now is your time to debug. 
6. Test on the real robot and debug your errors until you make it work properly

<img src="realrobot_follow_wall.gif"/>

# PART II: Services

### Create a ROS service that searches the wall

In this part, you will have to do three things:
1. Create a ROS service server that, when called, will make the robot go to the nearest wall
2. Modify the PART I program so that it calls the ROS service server before starting to follow the wall
3. Create a new launch file to launch both nodes

#### Step 1: Create the ROS service server

* Create a new ROS node that contains a service server named *find_wall*. The server uses a *FindWall* message that you must create.

In [ ]:
FindWall.srv

---
bool wallfound

When the service is called, the robot must do the following behavior:

1. Identify which of the laser rays is the shortest one. We assume that it is the one that is pointing to a wall.
2. Rotate the robot until the front of the robot is facing the wall. This can be done by rotating the robot until the ray 0 is the smaller one.
3. Move the robot forward until the ray 0 is shorter than 0.3m.
4. Now rotate again the robot until ray number 270 of the laser ranges is pointing to the wall.
5. At this point, we consider that the robot is placed to start moving along the wall
6. Return the service message with a *True*.

**How to test the service**

* Launch the service server node
* Use the terminal to test the service with a service call command

In [ ]:
rosservice call find_wall

This should start the robot movement looking for the wall.

<img src="prepare_for_wall_sim.gif"/>

#### Step 2: Modify the PART I program

* Add a service client to the node of the part I.
* Call the service before the control loop of that node, so the robot gets prepared autonomously before starting to follow the wall

**How to test it**

* Launch the service server node
* Launch the wall following node

This should start the robot movement (first the robot looks for the wall, then it starts to follow the wall).

#### Step 3: Create a new launch file

* Create a new launch file named *main.launch* that launches both nodes: first the service server node and then the wall following node.

This should start the robot movement (first the robot looks for the wall, then it starts to follow the wall).

**IMPORTANT**: Test everything on the simulation. When you have it working on the simulation, book a date in the *Real Robot Lab* and make it work there, too.

# PART III: Actions

### Create an action server that records odometry

In this part, you will have to:

1. Create an action server that, when called, starts recording odometry
2. Add a call to the action server from the wall follower node
3. Include the launch of the action server in the *main.launch*

#### Step 1: Create the action server

* Create a new ROS node that contains an action server named *record_odom*. The server uses a *OdomRecord.action* message, that you must create.

In [ ]:
OdomRecord.action

---
Point[] list_of_odoms
---
float32 current_total

* The server must start recording the (x,y,theta) odometry of the robot as a Point32, one measure every second. 
* As feedback, the action server must provide the total amount of meters that the robot has moved so far
* When the robot has done a complete lap, it has to finish and return the list of odometries recorded

**How to test the server**

* Launch the action server node
* Use the terminal to launch the *axclient* and test the action server 

#### Step 2: Modify the PART I program

* Add an action service client to the node of the part I.
* Call the action server before the control loop of that node, so the robot starts recording odometry before starting to follow the wall

**How to test it**

* Launch the action seerver node.
* Launch the *main.launch*. 

This should start the robot movement and start recording.

* Check the feedback topic of the action server to see how it is going.

#### Step 3: Create a new launch file

* Add to *main.launch* the launch of the action server node.

This should start the robot movement and odometry recording.

**IMPORTANT**: Test everything on the simulation. When you have it working on the simulation, book a date in the *Real Robot Lab* and make it work there, too.

# PART IV: Book a date to present your project on Youtube (optional)

An important part of your learning is to teach others what you have learned.

In order to get the most out of your learning, you should deliver a live presentation on your project. 

If you want to take this brave step, contact us at info@theconstructsim.com requesting a presentation and we will agree on the day and time.

**You will have to**:
1. Prepare your presentation. You will have to explain your project in 20 minutes: how you solved it and how it works.
2. On the agreed upon day, you will need to have a camera and mic ready.
3. We will contact you on the day and prepare the whole broadcast. We will handle everything, you only need to be ready to present.


The event will be broadcast on Youtube, so anybody around the world can attend and watch your presentation.

# PART V: Share your project on your LinkedIn or social networks

The rosject that you created with this project is a demonstration of your value as a ROS Developer.

You can share your rosject on any website to show the ROS projects you have worked on. This can help future employers hire you.

In order to share your rosject:
* Go to your list of rosjects
* Click on the *Share* button of your rosject to get the share link.

<img src="rosject.png" width="400"/>

* Then you can publish the link anywhere. Remember to use the *Permanent link*

<img src="sharelink.png" width="400"/>

## APPENDIX: How to connect to the real robot in RoBoX

Once you know the basics of the operation with a simulated version, it's time to use the real Turtlebot3 robot.

### You can book up to two 25-minute sessions per week

To make a booking, follow these simple steps:

### STEP 1: Book a session

In the main dashboard of The Construct, you can book a session by clicking on this icon:

<img src="reallab.png"/>

Then a booking page will appear where you can do two things:

* Check your already-done bookings

<img src="bookassession.png"/>

* Make a new booking

<img src="bookings.png"/>

You can also click on the **BOOK A SESSION** button to make a new one, which will take you to this menu, where you will select:

* The **Type of Robot** you want to book. For the moment, only Turtlebot3 is available.
* The Date and Time for your reservation. 

<img src="realrobotlab_notebooks/make_reservation_2.png"/>

The **dates and times shown are the ones available**. They come in **25-minute blocks**.

There is also a limitation on the number of bookings per week a user can make. 

This depends on your particular license and subscription.

<img src="realrobotlab_notebooks/make_reservation_3.png"/>

<img src="realrobotlab_notebooks/make_reservation_4.png"/>

### STEP 2: Launch the RoBoX-Turtlebot3 ROSJect 

To have the best experience, you need to launch the rosject with the simulation of Turtlebot3. Chances are that if you are reading this, you have already launched it.

When **one hour or less** is remaining until your booking session, a warning icon will be shown in **the top bar** of The Construct. This means that you have less than an hour to get ready for your session. You will also see the remaining time to your session.

<img src="realrobotlab_notebooks/booking_warnings1.png"/>

In case you are inside a rosject, the icon will appear at the bottom line of icons, like in the following image.

<img src="realrobotlab_notebooks/warning_4.png"/>

The orange dot will turn into a **red dot** when your booking session has started.

<img src="realrobotlab_notebooks/warning_2.png"/>

**Note**: Also, when you are inside a rosject, the icon for accessing the external camera will appear in the icons located at the bottom.

<img src="realrobotlab_notebooks/stream_icon.png"/>

Press that icon to get an overview of the robot space.

<img src="realrobotlab_notebooks/stream_image.png"/>

You should start the rosject that contains the ROS code that you want to use in the remote real robot lab now.

### STEP 3: Turn ON the RoBoX connection to the robot you booked

Once you are inside the rosject and your real robot session has started, click on the robot icon at the bottom. A popup with a toggle for **turning on the connection** will appear.

**WARNING: Nothing will appear unless you have a booking. So if you didn't make a booking in the dashboard, feel free to do so.**

Now you just have to click on the toggle to connect the desktop environment to the remote lab. This will start the connection process, which lasts for a few seconds. 

Once the connection process is finished, the **red dot will change to green** if succesfully connected. Now **any terminal you open will show the topics and services of the real robot**.

<img src="realrobotlab_notebooks/connect_robox.gif" width=600/>

You should be able to do the following command, getting a list of topics similar to this. The first time you issue a ROS command, it could take an extra 30 seconds to appear:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rostopic list

In [ ]:
/battery_state
/cmd_vel
/cmd_vel_rc100
/cv_camera/camera_info
/cv_camera/image_raw
/cv_camera/image_raw/compressed
/cv_camera/image_raw/compressed/parameter_descriptions
/cv_camera/image_raw/compressed/parameter_updates
/cv_camera/image_raw/compressedDepth
/cv_camera/image_raw/compressedDepth/parameter_descriptions
/cv_camera/image_raw/compressedDepth/parameter_updates
/cv_camera/image_raw/theora
/cv_camera/image_raw/theora/parameter_descriptions
/cv_camera/image_raw/theora/parameter_updates
/diagnostics
/firmware_version
/imu
/joint_states
/magnetic_field
/motor_power
/odom
/reset
/rosout
/rosout_agg
/rpms
/scan
/sensor_state
/sound
/tf
/tf_static
/version_info

**Note**: Take into account that you **will not be able to push git changes when turned on, because there is no interenet connection**.

So, to push any changes in your remote gits in the rosject, you will have to turn off the connection (or wait until your time is up).

You are now connected to the robot! So let's try moving the robot around and seeing the lasers and the camera in rviz.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

<span style='background :orange' > Move the robot around: Be careful to **NOT exceed a linear speed of 0.19 and angular of 0.49** because, otherwise, the node will be terminated for security reasons. </span>

In [ ]:
export TURTLEBOT3_MODEL=burger
roslaunch turtlebot3_teleop turtlebot3_teleop_key.launch

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rviz

You can then add an image and scan elements in the **base_link** frame, having something like this:

<img src="realrobotlab_notebooks/robox_demo_t3.gif" width=1200/>


## Ready for your programs!

If you have reached this point, it means that you have a proper connection with the remote real robot lab. **Now you can launch the programs of your project and see the results on a real robot**.